In [1]:
%load_ext autoreload
%autoreload 2
import var_selection
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(precision=4)

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

dataset = pd.read_csv('./data/happyness_data.csv', delimiter=' ')[['happy','money','sex','love','work']]
X = dataset[['money','sex','love','work']]
y = dataset[['happy']]
corrMat = X.corr()
print('Covariate sample correlation matrix:\n'+str(corrMat))
print('\nnumber of datapoints: {:d}'.format(X.shape[0]))

Covariate sample correlation matrix:
       money    sex  love   work
money  1.000  0.307 0.126  0.068
sex    0.307  1.000 0.047 -0.316
love   0.126  0.047 1.000  0.386
work   0.068 -0.316 0.386  1.000

number of datapoints: 39


#### Value are identical to those in the paper (page 16).

In [3]:
[Evals, Evecs] = np.linalg.eig(corrMat)
print('Eigenvalues of Covariate sample correlation matrix: \n'+str(Evals))

Eigenvalues of Covariate sample correlation matrix: 
[ 0.4405  0.7356  1.3468  1.4771]


#### Value are identical to those in the paper (page 16).

In [4]:
# convert pandas to numpy
if type(X) == pd.DataFrame:
    X = np.concatenate([X.as_matrix(), np.ones([X.shape[0],1])], axis = 1).astype(np.float64)
    y = y.as_matrix().astype(np.float64)

In [5]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# ordinary least squares solutions
OLS_β = np.dot( np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, y))

# standard deviation for ordinary least squares solution
# check out this: https://stats.stackexchange.com/questions/216335/standard-error-for-a-parameter-in-ordinary-least-squares
error = y - X.dot(OLS_β)
norm_sq_error = error.T.dot(error).flatten()[0]/36 # not sure about the 36 but this way it is closest to the paper
OLS_β_var = norm_sq_error * np.linalg.inv(np.dot(X.T, X))
OLS_β_std =  np.sqrt(norm_sq_error * np.atleast_2d(np.diag(OLS_β_var)[:-1]).T)

# priors for τ_star
delta_happiness = 4.
delta_money = 50.
delta_sex = 0.5
delta_love = 1.
delta_work = 2.
τ_star = np.atleast_2d(delta_happiness / np.array([delta_money,delta_sex,delta_love,delta_work])).T

stats = np.concatenate([OLS_β[:-1], OLS_β_std, τ_star], axis = 1)
stats = pd.DataFrame(stats, columns = ['OLS coefficient','OLS standard error', 'prior std τ*'], index = ['money','sex','love','work'])
print(stats)

       OLS coefficient  OLS standard error  prior std τ*
money          0.00958             0.00521       0.08000
sex           -0.14901             0.41836       8.00000
love           1.91928             0.29533       4.00000
work           0.47608             0.19931       2.00000


#### These values are very similar to those in the paper (page 16).  

In [6]:
# Formulas found here: https://pdfs.semanticscholar.org/3ace/886849dd48eb911b0491d70ef3ec197f9234.pdf
SS_reg = (OLS_β.T.dot(X.T.dot(y)) - (1./X.shape[0])*(np.ones_like(y).T.dot(y))**2)[0,0]
SS_total = (y.T.dot(y) - (1./X.shape[0])*(np.ones_like(y).T.dot(y))**2)[0,0]
print('multiple correlation coef: {:.3f}'.format(SS_reg/SS_total))

multiple correlation coef: 0.710


#### According to the paper the prior mean of the multiple correlation coefficient is 0.75. (?)

In [ ]:
%load_ext autoreload
%autoreload 2
import var_selection

for pp in [0.2, 0.5, 0.8]:
    for tauFactor in [0.5, 1., 2.]:
        
        print("Running: p = {}, tauFactor = {}".format(pp, tauFactor))
        
        # define subjective parameters
        conf = dict()
        conf["β"] = np.array([0., 0., 0., 0., 0.], dtype='float')
        conf["ϵ_sq"] = 2.5**2
        conf["υ"] = np.ones([5,1]) * np.infty
        conf["λ"] = np.concatenate([np.zeros([4,1]), -np.infty * np.ones([1,1])])
        conf["ν"] = 0.01
        conf["p"] = np.array([pp, pp, pp, pp, 0], dtype='float')
        conf["τ"] = np.concatenate([tauFactor * τ_star, 9 * np.ones([1,1])])
        conf["iterations"] = 1000000

        # β_post, var_post, chain, probs, tmp_models
        results= var_selection.variable_selection(X, y, conf["β"], conf["ϵ_sq"], conf["p"], conf["τ"], conf["ν"], 
                                                  conf["υ"], conf["λ"], conf["iterations"], verbose = 100000)
        
        pickle.dump(file = open("./results/1000000/p_"+str(conf["p"])+"_tau_"+str(conf["τ"].flatten())+".pickle", 'wb'), 
                    obj=[results,conf])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running: p = 0.2, tauFactor = 0.5
0th iteration
Error: 2473.8380233711896
β: [ 0.0814  0.      0.4576  1.2063  3.9626]
σ²: 1.4063778163852128e+113, σ: 3.7501704179746454e+56



/home/soren/Studium/MonteCarlo/mcmc_proj7/var_selection.py:37: RuntimeWarning: overflow encountered in exp
  bf = np.exp(β_bar**2/(2*var_star) - β_old**2/(2*τ**2))


In [ ]:
p02t05 = "p_[ 0.2  0.2  0.2  0.2  0. ]_tau_[ 0.04  4.    2.    1.    9.  ].pickle"
p02t1 = "p_[ 0.2  0.2  0.2  0.2  0. ]_tau_[ 0.08  8.    4.    2.    9.  ].pickle"
p02t2 = "p_[ 0.2  0.2  0.2  0.2  0. ]_tau_[  0.16  16.     8.     4.     9.  ].pickle"

#[results, config] = pickle.load(file = open("./results/"+p02t2, 'rb'))